In [4]:
# Import libraries
from google.cloud import aiplatform

In [7]:
!pip install google-cloud-aiplatform


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
# Initialize connection
aiplatform.init(location='us-central1')

In [ ]:
job = aiplatform.CustomContainerTrainingJob(
    display_name='custom_training_sdk',
    container_uri='us-central1-docker.pkg.dev/mdepew-assets/ml-repo/supersimple:latest',
    staging_bucket='mdepew-assets'
)

job.run(
    replica_count=1, 
    machine_type='n1-standard-4',
    args=['--dataset_dir=gs://mdepew-assets/dataset.csv']

)


In [12]:

from google.cloud import aiplatform
from google.cloud.aiplatform import hyperparameter_tuning as hpt

# The spec of the worker pools including machine type and Docker image
# Be sure to replace PROJECT_ID in the `image_uri` with your project.

worker_pool_specs = [{
    "machine_spec": {
        "machine_type": "n1-standard-4",
    },
    "replica_count": 1,
    "container_spec": {
        "image_uri": "us-central1-docker.pkg.dev/mdepew-assets/ml-repo/supersimple:v1",
    "args": [
        "python", 
        "-m", 
        "trainer.task", 
        "--dataset_dir", 
        "gs://mdepew-assets/dataset.csv",
        "--model_dir",
        "gs://mdepew-assets/model",
        "--max_depth",
        "1",
        "--learning_rate",
        "0.1"
        
    ]}
}]


# Dictionary representing metrics to optimize.
# The dictionary key is the metric_id, which is reported by your training job,
# And the dictionary value is the optimization goal of the metric.
metric_spec={'accuracy':'maximize'}

# Dictionary representing parameters to optimize.
# The dictionary key is the parameter_id, which is passed into your training
# job as a command line argument,
# And the dictionary value is the parameter specification of the metric.
parameter_spec = {
    "learning_rate": hpt.DoubleParameterSpec(min=0.001, max=1, scale="log"),
    "max_depth": hpt.IntegerParameterSpec(min=1, max=128, scale="linear")
    
}

# Replace YOUR_BUCKET
custom_job= aiplatform.CustomJob(display_name='hptune-sdk-job',
                              worker_pool_specs=worker_pool_specs,
                              staging_bucket='gs://mdepew-assets')


hp_job = aiplatform.HyperparameterTuningJob(
    display_name='tune-sdk-job',
    custom_job=custom_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=15,
    parallel_trial_count=3)

hp_job.run()

Creating HyperparameterTuningJob
HyperparameterTuningJob created. Resource name: projects/194822035697/locations/us-central1/hyperparameterTuningJobs/3976749629986308096
To use this HyperparameterTuningJob in another session:
hpt_job = aiplatform.HyperparameterTuningJob.get('projects/194822035697/locations/us-central1/hyperparameterTuningJobs/3976749629986308096')
View HyperparameterTuningJob:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/3976749629986308096?project=194822035697
HyperparameterTuningJob projects/194822035697/locations/us-central1/hyperparameterTuningJobs/3976749629986308096 current state:
JobState.JOB_STATE_RUNNING
HyperparameterTuningJob projects/194822035697/locations/us-central1/hyperparameterTuningJobs/3976749629986308096 current state:
JobState.JOB_STATE_RUNNING
HyperparameterTuningJob projects/194822035697/locations/us-central1/hyperparameterTuningJobs/3976749629986308096 current state:
JobState.JOB_STATE_RUNNING
HyperparameterTuningJ

RetryError: Deadline of 120.0s exceeded while calling target function, last exception: 503 recvmsg:Connection reset by peer